In [1]:
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

#import dataset

#download from external sources
triplets_file = 'https://static.turi.com/datasets/millionsong/10000.txt'
user_df = pd.read_table(triplets_file,header=None)
user_df.columns = ['user_id', 'song_id', 'listen_count']

metadata_file = 'https://static.turi.com/datasets/millionsong/song_data.csv'
song_df =  pd.read_csv(metadata_file)

In [2]:
# drop duplicates of song file since songs can be on multiple albums

song_df = song_df.drop_duplicates(subset=['song_id'])

In [3]:
# add column of song and artist to song df

song_df['song'] = song_df['title'] + " - " + song_df['artist_name']
song_df.head()
print(len(song_df))


999056


In [4]:
# merge to get dataset of user and song data
user_song_df = pd.merge(user_df, song_df, on='song_id', how='inner')
user_song_df.to_csv('user_song_data.csv', index=False)
len(user_song_df)
print(user_song_df.song_id.nunique())

10000


In [5]:
import sys
import spotipy
import spotipy.util as util
import pandas as pd


cid ="0d262097ee284912b168027f224d160a" 
secret = "9d1d17eac2034913883c828fcddd83f8"


scope = 'user-library-read'

if len(sys.argv) > 1:
    username = sys.argv[1]
else:
    print("Usage: %s username" % (sys.argv[0],))
    sys.exit()
    
token = util.prompt_for_user_token(username,scope,client_id=cid,
                                   client_secret=secret, redirect_uri='http://localhost/')

sp = spotipy.Spotify(auth=token)

In [6]:
def get_playlist_tracks(playlist):
        results = sp.user_playlist_tracks(username, playlist_id=playlist, fields='items(track(id,name,uri, artists)), next')
        tracks = results['items']
        while results['next']:
            results = sp.next(results)
            tracks.extend(results['items'])
        return tracks

Used http://www.playlist-converter.net to create spotify playlists of the songs found in our data set.

In [7]:
playlists = ['0Etj3HiMrDsr9bvpsNAgQ9', '0quHjtBflxWKehVf29FEG9', '1EnvIDy7VipQJhUYWI422i', '7zlt8VfE9E9qQ14CXMu5m7','1LZOokmdSBeRcezY0PpwrD',
            '1PHRnM4gbGmSXin0OjeYoV', '3dUdmbk5hjxRF7FcZHjgZD', '5QOxyTdXD8l7DGLu4w0eWl', '1BVdGBUd8XoPWavyE7bpgh', '1auFS66qn4dNVjPMIFQm5A',
            '7lpZsTYd2vjPec4I0QSqjT', '18FvbKUfcXkxiBxx9GlKci', '0lolsa13ZFcQ336KHADyXe', '6LkYNZGGX7LIamOFzR9H6J', '1wb4qVkJvJ6zCNO06B2pWj',
            '0WX1qkMuqxGcTVqcjElS9R','1qGgZp8okN3WNE4N1kcEkL','2GDEzDQduWvFYzVIOPccTL']

In [8]:
t_tracks = []
for playlist in playlists:
    tracks = get_playlist_tracks(playlist)
    tracks = [x['track'] for x in tracks]
    t_tracks.extend(tracks)
    

In [9]:
spotify_tracks = pd.DataFrame(t_tracks)
spotify_tracks = spotify_tracks.drop_duplicates(subset = ['id'])
print(spotify_tracks.head())
print(len(spotify_tracks))

                                             artists                      id  \
0  [{'external_urls': {'spotify': 'https://open.s...  3Fgk17TfdjXWFroJy29pXH   
1  [{'external_urls': {'spotify': 'https://open.s...  4fzsfWzRhPawzqhX8Qt9F3   
2  [{'external_urls': {'spotify': 'https://open.s...  4rpNjW4HXxh8o5UwS7UoWL   
3  [{'external_urls': {'spotify': 'https://open.s...  5OQsiBsky2k2kDKy2bX2eT   
4  [{'external_urls': {'spotify': 'https://open.s...  2mycbyJ4xTqySpNZz0t1ux   

                             name                                   uri  
0  Entre Dos Aguas - Instrumental  spotify:track:3Fgk17TfdjXWFroJy29pXH  
1                        Stronger  spotify:track:4fzsfWzRhPawzqhX8Qt9F3  
2                  Constellations  spotify:track:4rpNjW4HXxh8o5UwS7UoWL  
3                    Learn to Fly  spotify:track:5OQsiBsky2k2kDKy2bX2eT  
4                   Paper Gangsta  spotify:track:2mycbyJ4xTqySpNZz0t1ux  
7458


In [10]:
# turn artist column values from list to dictionaries (extract first value of list)
spotify_tracks['artists'] = [x[0] for x in spotify_tracks['artists']]

In [11]:
# extract artist name as new column
spotify_tracks['artist_name'] = spotify_tracks['artists'].apply(lambda x: x.get('name'))

# drop artists column
spotify_tracks = spotify_tracks.drop(['artists'], axis = 1)

In [12]:
spotify_tracks.head()
print(len(spotify_tracks))

7458


In [13]:
# lower case names 
user_song_df['title'] = user_song_df['title'].str.lower()
user_song_df['artist_name'] = user_song_df['artist_name'].str.lower() 

spotify_tracks['name'] = spotify_tracks['name'].str.lower()
spotify_tracks['artist_name'] = spotify_tracks['artist_name'].str.lower()


# merge data frame by song names
merged_df = pd.merge(user_song_df, spotify_tracks, left_on=('title', 'artist_name'), right_on=('name','artist_name'))

In [14]:
len(merged_df)


1143076

In [15]:
merged_df.head(50)

,user_id,song_id,listen_count,title,release,artist_name,year,song,id,name,uri
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1,stronger,Graduation,kanye west,2007,Stronger - Kanye West,4fzsfWzRhPawzqhX8Qt9F3,stronger,spotify:track:4fzsfWzRhPawzqhX8Qt9F3
1,9fba771d9731561eba47216f6fbfc0023d88641b,SOBXHDL12A81C204C0,19,stronger,Graduation,kanye west,2007,Stronger - Kanye West,4fzsfWzRhPawzqhX8Qt9F3,stronger,spotify:track:4fzsfWzRhPawzqhX8Qt9F3
2,85952991b8e3ca5803a08b0b2f9c6d71abf9bb5b,SOBXHDL12A81C204C0,1,stronger,Graduation,kanye west,2007,Stronger - Kanye West,4fzsfWzRhPawzqhX8Qt9F3,stronger,spotify:track:4fzsfWzRhPawzqhX8Qt9F3
3,537340ff896dea11328910013cfe759413e1eeb3,SOBXHDL12A81C204C0,2,stronger,Graduation,kanye west,2007,Stronger - Kanye West,4fzsfWzRhPawzqhX8Qt9F3,stronger,spotify:track:4fzsfWzRhPawzqhX8Qt9F3
4,8fce200f3912e9608e3b1463cdb9c3529aab5c08,SOBXHDL12A81C204C0,2,stronger,Graduation,kanye west,2007,Stronger - Kanye West,4fzsfWzRhPawzqhX8Qt9F3,stronger,spotify:track:4fzsfWzRhPawzqhX8Qt9F3
5,c24ec42f0e449ff39a95a01f0795f833b898f71b,SOBXHDL12A81C204C0,1,stronger,Graduation,kanye west,2007,Stronger - Kanye West,4fzsfWzRhPawzqhX8Qt9F3,stronger,spotify:track:4fzsfWzRhPawzqhX8Qt9F3
6,7b544bb0c02878033e4e4eea4df0a10466a71405,SOBXHDL12A81C204C0,2,stronger,Graduation,kanye west,2007,Stronger - Kanye West,4fzsfWzRhPawzqhX8Qt9F3,stronger,spotify:track:4fzsfWzRhPawzqhX8Qt9F3
7,d253ea7a210de16f723a165fd8525e8fc73f3939,SOBXHDL12A81C204C0,5,stronger,Graduation,kanye west,2007,Stronger - Kanye West,4fzsfWzRhPawzqhX8Qt9F3,stronger,spotify:track:4fzsfWzRhPawzqhX8Qt9F3
8,f904f6ed9ac7c35f4771c69a9ef38e797e4e4baf,SOBXHDL12A81C204C0,1,stronger,Graduation,kanye west,2007,Stronger - Kanye West,4fzsfWzRhPawzqhX8Qt9F3,stronger,spotify:track:4fzsfWzRhPawzqhX8Qt9F3
9,405d29f1c07646a27a3216a69d286fa080aab8e6,SOBXHDL12A81C204C0,1,stronger,Graduation,kanye west,2007,Stronger - Kanye West,4fzsfWzRhPawzqhX8Qt9F3,stronger,spotify:track:4fzsfWzRhPawzqhX8Qt9F3


In [16]:
merged_df.song_id.nunique()

5763

In [17]:
# get list of unique spotify tracks in merged_df to extract audio features

unique_tracks = merged_df.id.unique()

In [18]:
# limit of 100 tracks at a time, cut into batches

import timeit

# measuring the time
start = timeit.default_timer()

# empty list, batchsize and the counter for None results
features = []
batchsize = 100
None_counter = 0

for i in range(0,len(unique_tracks),batchsize):
    batch = unique_tracks[i:i+batchsize]
    feature_results = sp.audio_features(batch)
    for i, t in enumerate(feature_results):
        if t == None:
            None_counter = None_counter + 1
        else:
            features.append(t)
            
print('Number of tracks where no audio features were available:',None_counter)

stop = timeit.default_timer()
print ('Time to run this code (in seconds):',stop - start)

Number of tracks where no audio features were available: 0
Time to run this code (in seconds): 9.72523558999999


In [19]:
features_df = pd.DataFrame(features)
print(features_df.head())

   acousticness                                       analysis_url  \
0      0.005640  https://api.spotify.com/v1/audio-analysis/4fzs...   
1      0.470000  https://api.spotify.com/v1/audio-analysis/4rpN...   
2      0.000018  https://api.spotify.com/v1/audio-analysis/5OQs...   
3      0.050300  https://api.spotify.com/v1/audio-analysis/2myc...   
4      0.002030  https://api.spotify.com/v1/audio-analysis/1vzi...   

   danceability  duration_ms  energy                      id  \
0         0.617       311867   0.717  4fzsfWzRhPawzqhX8Qt9F3   
1         0.719       201653   0.299  4rpNjW4HXxh8o5UwS7UoWL   
2         0.465       235293   0.919  5OQsiBsky2k2kDKy2bX2eT   
3         0.755       263360   0.839  2mycbyJ4xTqySpNZz0t1ux   
4         0.502       256440   0.934  1vziI0NCJKrqKiG94COHBa   

   instrumentalness  key  liveness  loudness  mode  speechiness    tempo  \
0          0.000000   10     0.408    -7.858     0       0.1530  103.992   
1          0.000034    1     0.111   -12.4

In [20]:
merged_df2 = pd.merge(merged_df, features_df, on = 'id', how = 'left')
merged_df2.head()

,user_id,song_id,listen_count,title,release,artist_name,year,song,id,name,...,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri_y,valence
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1,stronger,Graduation,kanye west,2007,Stronger - Kanye West,4fzsfWzRhPawzqhX8Qt9F3,stronger,...,0.408,-7.858,0,0.153,103.992,4,https://api.spotify.com/v1/tracks/4fzsfWzRhPaw...,audio_features,spotify:track:4fzsfWzRhPawzqhX8Qt9F3,0.49
1,9fba771d9731561eba47216f6fbfc0023d88641b,SOBXHDL12A81C204C0,19,stronger,Graduation,kanye west,2007,Stronger - Kanye West,4fzsfWzRhPawzqhX8Qt9F3,stronger,...,0.408,-7.858,0,0.153,103.992,4,https://api.spotify.com/v1/tracks/4fzsfWzRhPaw...,audio_features,spotify:track:4fzsfWzRhPawzqhX8Qt9F3,0.49
2,85952991b8e3ca5803a08b0b2f9c6d71abf9bb5b,SOBXHDL12A81C204C0,1,stronger,Graduation,kanye west,2007,Stronger - Kanye West,4fzsfWzRhPawzqhX8Qt9F3,stronger,...,0.408,-7.858,0,0.153,103.992,4,https://api.spotify.com/v1/tracks/4fzsfWzRhPaw...,audio_features,spotify:track:4fzsfWzRhPawzqhX8Qt9F3,0.49
3,537340ff896dea11328910013cfe759413e1eeb3,SOBXHDL12A81C204C0,2,stronger,Graduation,kanye west,2007,Stronger - Kanye West,4fzsfWzRhPawzqhX8Qt9F3,stronger,...,0.408,-7.858,0,0.153,103.992,4,https://api.spotify.com/v1/tracks/4fzsfWzRhPaw...,audio_features,spotify:track:4fzsfWzRhPawzqhX8Qt9F3,0.49
4,8fce200f3912e9608e3b1463cdb9c3529aab5c08,SOBXHDL12A81C204C0,2,stronger,Graduation,kanye west,2007,Stronger - Kanye West,4fzsfWzRhPawzqhX8Qt9F3,stronger,...,0.408,-7.858,0,0.153,103.992,4,https://api.spotify.com/v1/tracks/4fzsfWzRhPaw...,audio_features,spotify:track:4fzsfWzRhPawzqhX8Qt9F3,0.49


In [21]:
len(merged_df2)

1143076

In [22]:
merged_df2.song_id.nunique()

5763

In [23]:
len(user_df)

2000000

In [24]:
# export dataframe

merged_df2.to_csv('User_SongFeatures_data.csv', index=False)